In [46]:
pip install pingouin

In [47]:

from scipy.stats import kruskal
import scikit_posthocs as sp
import numpy as np
import pingouin as pg
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats import friedmanchisquare
import pandas as pds
from scipy.stats import kruskal
import pingouin as pg



url = # Load the CSV file
df = pd.read_csv(url)
columns_to_check = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q16', 'Q17']


In [48]:

for col in columns_to_check:

    # Shapiro-Wilk test for normality
    stat, p = shapiro(df[col].dropna())  # dropna() is used to remove any NaN values

    if p > 0.05:
        print(f"{col} seems to have a normal distribution (p={p:.2f})")
    else:
        print(f"{col} does not seem to have a normal distribution (p={p:.2f})")

Q1 does not seem to have a normal distribution (p=0.00)
Q2 does not seem to have a normal distribution (p=0.00)
Q3 does not seem to have a normal distribution (p=0.00)
Q4 does not seem to have a normal distribution (p=0.00)
Q5 does not seem to have a normal distribution (p=0.00)
Q6 does not seem to have a normal distribution (p=0.00)
Q7 does not seem to have a normal distribution (p=0.00)
Q8 does not seem to have a normal distribution (p=0.00)
Q10 does not seem to have a normal distribution (p=0.00)
Q11 does not seem to have a normal distribution (p=0.00)
Q12 does not seem to have a normal distribution (p=0.00)
Q13 does not seem to have a normal distribution (p=0.00)
Q14 does not seem to have a normal distribution (p=0.00)
Q16 does not seem to have a normal distribution (p=0.00)
Q17 does not seem to have a normal distribution (p=0.00)


In [49]:

means = df.groupby('experiment_type')[columns_to_check].mean().T.round(2)
std_devs = df.groupby('experiment_type')[columns_to_check].std().T.round(2)

combined = means.astype(str) + " (" + std_devs.astype(str) + ")"


print("Combined Means and Standard Deviations:\n", combined)


Combined Means and Standard Deviations:
 experiment_type            0            1            2            3  \
Q1               5.64 (1.48)  5.51 (1.56)  5.81 (1.11)  5.92 (1.25)   
Q2               5.21 (1.88)  5.44 (1.63)  5.55 (1.34)  5.65 (1.29)   
Q3               4.85 (2.08)   4.6 (2.01)  5.13 (2.03)  4.86 (1.82)   
Q4                5.6 (1.44)  5.66 (1.71)  5.68 (1.37)   5.88 (1.4)   
Q5               5.52 (1.44)  5.66 (1.27)  5.59 (1.27)  5.65 (1.24)   
Q6               6.38 (1.22)  6.04 (1.36)  6.17 (1.05)   6.3 (1.15)   
Q7                6.3 (1.07)  6.34 (1.15)  6.35 (1.07)   6.5 (0.89)   
Q8               6.33 (1.28)  6.12 (1.31)  6.45 (0.99)  6.26 (1.21)   
Q10              5.94 (1.05)   5.84 (1.3)  5.46 (1.36)  5.65 (1.43)   
Q11              4.44 (1.26)   4.69 (1.3)  4.49 (1.36)  4.41 (1.49)   
Q12              5.65 (1.43)   5.6 (1.52)  5.91 (1.19)  5.84 (1.28)   
Q13              5.54 (1.54)  5.53 (1.41)  5.97 (1.02)  5.98 (1.38)   
Q14              5.72 (1.71)  5.77 (

In [50]:
p_values_dict = {}

for column in columns_to_check:
    samples = [group[column].dropna().values for _, group in df.groupby('experiment_type')]
    _, p = kruskal(*samples)
    p_values_dict[column] = p



means['p-value'] = pd.Series(p_values_dict)

print(means)

experiment_type     0     1     2     3     4     5   p-value
Q1               5.64  5.51  5.81  5.92  5.50  5.91  0.214190
Q2               5.21  5.44  5.55  5.65  5.54  5.99  0.168404
Q3               4.85  4.60  5.13  4.86  4.58  5.18  0.176207
Q4               5.60  5.66  5.68  5.88  5.79  6.01  0.285408
Q5               5.52  5.66  5.59  5.65  5.39  5.68  0.850081
Q6               6.38  6.04  6.17  6.30  5.98  6.35  0.054971
Q7               6.30  6.34  6.35  6.50  6.36  6.49  0.652556
Q8               6.33  6.12  6.45  6.26  6.34  6.65  0.118194
Q10              5.94  5.84  5.46  5.65  5.66  5.99  0.047373
Q11              4.44  4.69  4.49  4.41  4.72  4.70  0.464500
Q12              5.65  5.60  5.91  5.84  5.62  6.02  0.387362
Q13              5.54  5.53  5.97  5.98  5.76  6.12  0.018212
Q14              5.72  5.77  6.06  5.99  5.75  6.12  0.781203
Q16              6.27  6.23  6.41  6.38  6.39  6.49  0.393729
Q17              6.38  6.27  6.58  6.34  6.50  6.50  0.295850


In [61]:

def posthoc(df, column_name):
    df_column = df[['experiment_type', column_name]]

    experiment_groups = []
    for experiment_type, group_data in df_column.groupby('experiment_type'):
        experiment_groups.append(group_data[column_name].dropna())

    # Run Kruskal-Wallis test
    _, p_value = kruskal(*experiment_groups)
    print(f"Kruskal-Wallis p-value for {column_name}:", p_value)

    # post hoc - pairwise_ttests
    posthoc_comp = pg.pairwise_tests(data=df_column, dv=column_name, between='experiment_type')
    print(f"Post Hoc Test for {column_name}:\n", posthoc_comp)

In [62]:
posthoc(df, 'Q6')

Kruskal-Wallis p-value for Q6: 0.054971014518015844
Post Hoc Test for Q6:
            Contrast  A  B  Paired  Parametric         T         dof  \
0   experiment_type  0  1   False        True  1.668252  152.121035   
1   experiment_type  0  2   False        True  1.198244  155.056324   
2   experiment_type  0  3   False        True  0.442342  158.674812   
3   experiment_type  0  4   False        True  1.852479  149.842501   
4   experiment_type  0  5   False        True  0.173626  158.863274   
5   experiment_type  1  2   False        True -0.653406  142.818026   
6   experiment_type  1  3   False        True -1.294612  148.826471   
7   experiment_type  1  4   False        True  0.274914  153.722640   
8   experiment_type  1  5   False        True -1.532479  149.697761   
9   experiment_type  2  3   False        True -0.760886  155.298364   
10  experiment_type  2  4   False        True  0.911870  139.166586   
11  experiment_type  2  5   False        True -1.037173  154.931206   
12

In [63]:
posthoc(df, 'Q10')

Kruskal-Wallis p-value for Q10: 0.04737292326556349
Post Hoc Test for Q10:
            Contrast  A  B  Paired  Parametric         T         dof  \
0   experiment_type  0  1   False        True  0.498939  146.376531   
1   experiment_type  0  2   False        True  2.470833  145.293821   
2   experiment_type  0  3   False        True  1.453186  145.004691   
3   experiment_type  0  4   False        True  1.444366  149.265252   
4   experiment_type  0  5   False        True -0.267847  153.121685   
5   experiment_type  1  2   False        True  1.795008  152.865332   
6   experiment_type  1  3   False        True  0.890290  154.445573   
7   experiment_type  1  4   False        True  0.859567  154.999996   
8   experiment_type  1  5   False        True -0.699548  154.400997   
9   experiment_type  2  3   False        True -0.849606  155.856455   
10  experiment_type  2  4   False        True -0.933871  155.864105   
11  experiment_type  2  5   False        True -2.517931  154.695188   
1

In [64]:
posthoc(df, 'Q13')

Kruskal-Wallis p-value for Q13: 0.018212412278751044
Post Hoc Test for Q13:
            Contrast  A  B  Paired  Parametric         T         dof  \
0   experiment_type  0  1   False        True  0.045736  155.776353   
1   experiment_type  0  2   False        True -2.087756  139.278514   
2   experiment_type  0  3   False        True -1.874472  157.443266   
3   experiment_type  0  4   False        True -0.921513  158.858885   
4   experiment_type  0  5   False        True -2.705941  148.718065   
5   experiment_type  1  2   False        True -2.233500  138.268793   
6   experiment_type  1  3   False        True -1.987866  154.409198   
7   experiment_type  1  4   False        True -0.998027  154.989741   
8   experiment_type  1  5   False        True -2.866700  147.297101   
9   experiment_type  2  3   False        True -0.003331  145.517477   
10  experiment_type  2  4   False        True  1.051389  140.551629   
11  experiment_type  2  5   False        True -0.866809  154.276329   
